## ASA Data Challenge: 
### Analysis of LAPD Call Service Data from 2022

In [110]:
import pandas as pd
import matplotlib.pyplot as plt
from datascience import *
import numpy as np
import random
from sodapy import Socrata

### Collect Data using Socrata API which provides access to publicized government datasets 

In [111]:
client = Socrata("data.lacity.org", None)

In [118]:
import time

all_results = []
batch_size = 1000
offset = 0

while True:
    try:
        batch = client.get("u6ri-98iw", limit=batch_size, offset=offset)
        if not batch:
            break
        all_results.extend(batch)
        offset += batch_size
    except Exception as e:
        print(f"Error: {e}. Retrying in 5 seconds...")
        time.sleep(5)  # Wait before retrying


Error: HTTPSConnectionPool(host='data.lacity.org', port=443): Read timed out. (read timeout=10). Retrying in 5 seconds...
Error: HTTPSConnectionPool(host='data.lacity.org', port=443): Read timed out. (read timeout=10). Retrying in 5 seconds...


### Convert to Pandas df to view dataset description statistics

In [147]:
la24_results_df = pd.DataFrame.from_records(all_results)

la24_results_df.head(5)

#la24_results_df.describe()

,incident_number,area_occ,dispatch_date,dispatch_time,call_type_code,call_type_text,rpt_dist
0,PD22012600000212,Outside,2022-01-26T00:00:00.000,01:29:56,006,CODE 6,NaN
1,PD22012600000211,Outside,2022-01-26T00:00:00.000,01:29:50,902,TRAFFIC STOP,NaN
2,PD22012600000210,Outside,2022-01-26T00:00:00.000,01:29:23,006,CODE 6,NaN
3,PD22012600000203,Central,2022-01-26T00:00:00.000,01:29:12,4591I,BFV INVEST,0162
4,PD22012600000208,Outside,2022-01-26T00:00:00.000,01:28:54,902,TRAFFIC STOP,NaN


### Convert to Datascience Table() due to familiarity 

In [122]:
la24_results_table = Table().from_df(la24_results_df)

la24_results_table.show(20)

incident_number,area_occ,dispatch_date,dispatch_time,call_type_code,call_type_text,rpt_dist
PD22012600000212,Outside,2022-01-26T00:00:00.000,01:29:56,006,CODE 6,nan
PD22012600000211,Outside,2022-01-26T00:00:00.000,01:29:50,902,TRAFFIC STOP,nan
PD22012600000210,Outside,2022-01-26T00:00:00.000,01:29:23,006,CODE 6,nan
PD22012600000203,Central,2022-01-26T00:00:00.000,01:29:12,4591I,BFV INVEST,0162
PD22012600000208,Outside,2022-01-26T00:00:00.000,01:28:54,902,TRAFFIC STOP,nan
PD22012600000207,Outside,2022-01-26T00:00:00.000,01:28:38,902,TRAFFIC STOP,nan
PD22012600000204,Outside,2022-01-26T00:00:00.000,01:26:43,006,CODE 6,nan
PD22060700002534,Pacific,2022-06-07T00:00:00.000,14:24:44,7203,FD,1455
PD22012600000198,Rampart,2022-01-26T00:00:00.000,01:25:39,245PFJ,POSS SHOTS FIRED J/O,0247
PD22012600000194,Wilshire,2022-01-26T00:00:00.000,01:25:14,507O,OTHER,0706


### Get distinct area names and counts of reports from each area, scale them down, provide unique colors for each, and provide their general coordinates according to LAPD Station mapping and general information

In [123]:
areas = la24_results_table.group("area_occ").column(0)

counts = la24_results_table.group("area_occ").column(1)

counts_divided = counts/100

colors = [f"#{random.randint(0, 0xFFFFFF):06x}" for _ in areas]

la24_results_latitudes = [33.9458, 34.0443, 34.2635, 34.2723, 33.7734, 
 34.0498, 34.1016, 34.2728, 34.187, 34.0224, 
 34.1222, 34.0536, None, 33.9935, 34.0622, 
 33.9461, 34.0101, 34.2154, 34.1867, 34.0452, 
 34.187, 34.0622]

la24_results_longitudes = [-118.2917, -118.2528, -118.5075, -118.4148, -118.262, 
 -118.212, -118.3296, -118.4482, -118.3861, -118.2448, 
 -118.2096, -118.277, None, -118.4561, -118.2766, 
 -118.2397, -118.3091, -118.597, -118.4493, -118.4426, 
 -118.5379, -118.308]

la24_data_table = Table().with_columns("Areas", areas, "Latitude", la24_results_latitudes, "Longitude", la24_results_longitudes, "colors", colors, "areas", counts_divided, "labels", [areas + ': ' + str(counts_divided) + ' (hundreds)' for areas, counts_divided in zip(areas, counts_divided)])

la24_data_table.show()

la24_results_table.group("call_type_text").sort("count", descending=True).show(20)

Areas,Latitude,Longitude,colors,areas,labels
77th Street,33.9458,-118.292,#76e5b3,587.22,77th Street: 587.22 (hundreds)
Central,34.0443,-118.253,#0ead7b,603.97,Central: 603.97 (hundreds)
Devonshire,34.2635,-118.507,#619ef4,416.36,Devonshire: 416.36 (hundreds)
Foothill,34.2723,-118.415,#7542e3,360.22,Foothill: 360.22 (hundreds)
Harbor,33.7734,-118.262,#498fd5,382.48,Harbor: 382.48 (hundreds)
Hollenbeck,34.0498,-118.212,#49328a,395.89,Hollenbeck: 395.89 (hundreds)
Hollywood,34.1016,-118.33,#f1cee0,493.6,Hollywood: 493.6 (hundreds)
Mission,34.2728,-118.448,#268799,450.64,Mission: 450.64 (hundreds)
N Hollywood,34.187,-118.386,#805453,469.47,N Hollywood: 469.47 (hundreds)
Newton,34.0224,-118.245,#8768f5,494.84,Newton: 494.84 (hundreds)


call_type_text,count
CODE 6,590350
TRAFFIC STOP,45422
MAN,42917
CODE 30 RINGER,29473
SUSP NOW,29379
902 TRAFFIC STOP,23072
TRESPASS SUSP,21197
SUSP,19117
AMB,19006
DOM VIOL,17823


### Map data for each area, size of circle is based on count and label includes area and the count in the hundreds

Note: 'Outside' is not an area and can be generalized into surrounding cities not covered by LAPD stations

### What is the 'Outside' Category for LA Area: 

Calabasas, Malibu, Simi Valley, Santa Clarita San Fernando, Burbank, Glendale, Pasadena, East LA, Hunington Park, Compton, Downey, Inglewood, Gardena, Carson, Long Beach, Torrance, Redondo, San Pedro, Lakewood, Southgate, Santa Montica, Culver City, Beverly Hills, give or take.

In [124]:
# Circle.map_table(
#     la24_data_table.where("Latitude", are.not_equal_to(None))
#                   .select('Latitude', 'Longitude', 'Areas', 'colors', 'areas', 'labels'),
#     clustered_marker=False
# )
# Generate the map
import folium

# Filter data where Latitude is not None
filtered_data = la24_data_table.where("Latitude", are.not_equal_to(None))

# Create a folium map centered on the average latitude and longitude
map_object = folium.Map(
    location=[filtered_data.column('Latitude').mean(), filtered_data.column('Longitude').mean()],
    zoom_start=12
)

# Scale factor to adjust circle sizes (adjust as needed)
scale_factor = 0.5

# Add circle markers for each row in the Data Science Table
for i in range(filtered_data.num_rows):
    count = filtered_data.column('areas')[i]/25  # Assume 'count' column exists
    folium.CircleMarker(
        location=[filtered_data.column('Latitude')[i], filtered_data.column('Longitude')[i]],  # Latitude and Longitude
        radius=max(5, count * scale_factor),  # Circle size based on count, minimum size of 5
        color=filtered_data.column('colors')[i],  # Circle color
        fill=True,
        fill_opacity=0.6,
        popup=f"Area: {filtered_data.column('Areas')[i]}<br>Label: {filtered_data.column('labels')[i]}<br>Count: {count}"  # Popup info
    ).add_to(map_object)

# Save the map as an HTML file
map_filename = "service_calls_map.html"
map_object.save(map_filename)

print(f"Map saved as {map_filename}")


Map saved as service_calls_map.html


### Group by area and call type and sort to get most common occurrence in LA area

In [128]:
la24_area_call = la24_results_table.group(["area_occ", "call_type_code"]).sort('count', descending=True)
la24_area_call.show(50)

area_occ,call_type_code,count
Outside,006,449970
Outside,902,40209
Mission,006,10983
Central,006,8787
Devonshire,006,8036
Wilshire,006,7869
Newton,006,7534
Van Nuys,006,7497
Southwest,006,6954
Pacific,006,6924


### Generate a bar chart for the top 20 reported calls in LA area

In [129]:
import matplotlib.pyplot as plt
import numpy as np

# Aggregate call counts by call type
call_type_counts = la24_area_call.group('call_type_code', sum).sort('count sum', descending=True)

# Limit to top N call types
top_n = 20
top_call_types = call_type_counts.take(np.arange(top_n))

# Extract labels and counts
labels = top_call_types.column('call_type_code')
counts = top_call_types.column('count sum')

# Sum the rest into "Others" if applicable
if call_type_counts.num_rows > top_n:
    others_count = sum(call_type_counts.take(np.arange(top_n, call_type_counts.num_rows)).column('count sum'))
    labels = np.append(labels, "Others")
    counts = np.append(counts, others_count)

# Create the bar chart
plt.figure(figsize=(12, 6))
bars = plt.bar(labels, counts)
plt.title(f"Top {top_n} Call Types in the Dataset")
plt.xlabel("Call Type")
plt.ylabel("Count")
plt.xticks(rotation=45, ha='right')  # Rotate labels for readability
plt.tight_layout()

# Add exact count labels above the bars
for bar, count in zip(bars, counts):
    plt.text(
        bar.get_x() + bar.get_width() / 2,  # Center the label on the bar
        bar.get_height(),  # Position it above the bar
        f"{int(count)}",  # Format the count as an integer
        ha='center', va='bottom', fontsize=8  # Align and style the label
    )

# Save the chart as a PNG file
filename = "overall_call_type_barchart.png"
plt.savefig(filename)
plt.close()  # Close the plot to avoid memory issues

print(f"Saved bar chart for top call types as {filename}")

Saved bar chart for top call types as overall_call_type_barchart.png


### Looks like a lot of Code 6's, which can be translated as an officer arriving on scene with no crime unfolding, and as a result, investigates and calls a code 6 when there seems to be no need for assistance.

### Let's get a PNG for each area, with the top 20 call types displayed along with an "other" category containing the rest. This can let us know what are the most common calls dependent on the area in LA

In [131]:
# Extract unique areas and call types
areas = np.unique(la24_area_call.column("area_occ"))
call_types = np.unique(la24_area_call.column("call_type_code"))

# Limit the number of call types to display per area
top_n = 20  # Adjust this for the desired number of categories

for area in areas:
    # Filter data for the current area
    area_data = la24_area_call.where("area_occ", area)

    # Sort call types by count and take the top N
    top_call_types = area_data.sort("count", descending=True).take(np.arange(top_n))

    # Prepare labels and counts for the top N call types
    labels = top_call_types.column("call_type_code")
    counts = top_call_types.column("count")

    # Sum the rest into "Others" if applicable
    if area_data.num_rows > top_n:
        others_count = sum(area_data.sort("count", descending=True).take(np.arange(top_n, area_data.num_rows)).column("count"))
        labels = np.append(labels, "Others")
        counts = np.append(counts, others_count)

    # Create the bar chart for the current area
    plt.figure(figsize=(12, 6))
    bars = plt.bar(labels, counts)
    plt.title(f"Top {top_n} Call Types for Area: {area}")
    plt.xlabel("Call Type")
    plt.ylabel("Count")
    plt.xticks(rotation=45, ha='right')  # Rotate labels for readability
    plt.tight_layout()

    # Add exact count labels above the bars
    for bar, count in zip(bars, counts):
        plt.text(
            bar.get_x() + bar.get_width() / 2,  # Center the label on the bar
            bar.get_height(),  # Position it above the bar
            f"{int(count)}",  # Format the count as an integer
            ha='center', va='bottom', fontsize=8  # Align and style the label
        )

    # Save the chart as a PNG file
    filename = f"top_{top_n}_call_types_{area.replace(' ', '_')}.png"
    plt.savefig(filename)
    plt.close()  # Close the plot to avoid memory issues
    print(f"Saved bar chart for area '{area}' as {filename}")

Saved bar chart for area '77th Street' as top_20_call_types_77th_Street.png
Saved bar chart for area 'Central' as top_20_call_types_Central.png
Saved bar chart for area 'Devonshire' as top_20_call_types_Devonshire.png
Saved bar chart for area 'Foothill' as top_20_call_types_Foothill.png
Saved bar chart for area 'Harbor' as top_20_call_types_Harbor.png
Saved bar chart for area 'Hollenbeck' as top_20_call_types_Hollenbeck.png
Saved bar chart for area 'Hollywood' as top_20_call_types_Hollywood.png
Saved bar chart for area 'Mission' as top_20_call_types_Mission.png
Saved bar chart for area 'N Hollywood' as top_20_call_types_N_Hollywood.png
Saved bar chart for area 'Newton' as top_20_call_types_Newton.png
Saved bar chart for area 'Northeast' as top_20_call_types_Northeast.png
Saved bar chart for area 'Olympic' as top_20_call_types_Olympic.png
Saved bar chart for area 'Outside' as top_20_call_types_Outside.png
Saved bar chart for area 'Pacific' as top_20_call_types_Pacific.png
Saved bar char

### Visualize reports over time througout the year

In [135]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import numpy as np

# Simulate the 'la24_results_table' data for demonstration purposes
data = {
    'dispatch_date': pd.date_range(start='2024-01-01', periods=100, freq='D'),
    'count': np.random.poisson(lam=50, size=100)  # Simulating daily service call counts
}

# Create a DataFrame to mimic 'la24_results_table.group('dispatch_date')'
la24_results_table_df = pd.DataFrame(data)

# Convert 'dispatch_date' to a pandas DataFrame for easier time-based grouping
dispatch_df = pd.DataFrame({
    'dispatch_date': la24_results_table_df['dispatch_date'],
    'count': la24_results_table_df['count']
})

# Ensure 'dispatch_date' is treated as a datetime object
dispatch_df['dispatch_date'] = pd.to_datetime(dispatch_df['dispatch_date'])

# Group data by month or week and sum the counts
# For weekly grouping, change 'M' to 'W'
grouped = dispatch_df.groupby(dispatch_df['dispatch_date'].dt.to_period('W'))['count'].sum()
grouped.index = grouped.index.to_timestamp()  # Convert period to timestamp

# Apply Gaussian smoothing to the counts for a smoother curve
smoothed_counts = gaussian_filter1d(grouped.values, sigma=2)

# Plot
plt.figure(figsize=(12, 6))
plt.plot(grouped.index, smoothed_counts, label="Smoothed Count")
plt.scatter(grouped.index, grouped.values, color='red', label="Original Counts", alpha=0.6)
plt.title("Number of Service Calls by Dispatch Date (Smoothed)")
plt.xlabel("Dispatch Date (Weekly)")
plt.ylabel("Number of Calls")
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels
plt.legend()
plt.tight_layout()

# Save the plot as a PNG file
plot_filename = "smoothed_dispatch_date_call_counts.png"
plt.savefig(plot_filename)
plt.close()

print(f"Plot saved as {plot_filename}")

Plot saved as smoothed_dispatch_date_call_counts.png


In [136]:
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import numpy as np
import nbformat

# Step 1: Identify the top 20 most common call types
top_n = 20
call_type_totals = la24_results_table.group('call_type_text').select('call_type_text', 'count').sort('count', descending=True)
top_call_types = call_type_totals.take(np.arange(top_n)).column('call_type_text')

# Step 2: Categorize call types into top 20 and "Others"
la24_results_table = la24_results_table.with_column(
    'call_type_category',
    la24_results_table.apply(lambda x: x if x in top_call_types else 'Others', 'call_type_text')
)

# Step 3: Group by dispatch_date and call_type_category
grouped_data = la24_results_table.group(['dispatch_date', 'call_type_category'])

# Step 4: Group data bi-weekly and prepare smoothed data for plotting
smoothed_data = []
for call_type in grouped_data.group('call_type_category').column('call_type_category'):
    call_type_data = grouped_data.where('call_type_category', call_type)
    
    # Convert dispatch_date to datetime for grouping
    dispatch_dates = pd.to_datetime(call_type_data.column('dispatch_date'))
    counts = np.array([int(x) if x != '' else 0 for x in call_type_data.column('count')])
    
    # Group bi-weekly
    biweekly_data = pd.DataFrame({'dispatch_date': dispatch_dates, 'count': counts})
    biweekly_grouped = biweekly_data.groupby(biweekly_data['dispatch_date'].dt.to_period('2W'))['count'].sum()
    biweekly_grouped.index = biweekly_grouped.index.to_timestamp()  # Convert periods to timestamps
    
    # Apply smoothing
    smoothed_counts = gaussian_filter1d(biweekly_grouped.values, sigma=2)
    smoothed_data.append({
        'dispatch_date': biweekly_grouped.index,
        'call_type_category': call_type,
        'smoothed_counts': smoothed_counts
    })

# Step 5: Plot each line on the same coordinate plane
plt.figure(figsize=(15, 10))

for data in smoothed_data:
    # Plot using logarithmic Y-axis scale
    plt.plot(data['dispatch_date'], np.log10(data['smoothed_counts'] + 1), label=data['call_type_category'])
    
    # Add labels showing raw counts only at the start of each month
    for x, y in zip(data['dispatch_date'], data['smoothed_counts']):
        if x.day == 1:  # Label only at the start of each month
            # Add an offset proportional to the count to space the labels
            offset = 0.2 * np.log10(max(y + 1, 2))
            plt.text(x, np.log10(y + 1) + offset, f"{int(y)}", fontsize=8, ha='center', va='bottom')

# Customize the plot
plt.title(f"Top {top_n} Call Types Over Time (Logarithmic Y-Axis)")
plt.xlabel("Dispatch Date")
plt.ylabel("Log10(Number of Calls + 1)")
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.legend(title="Call Type", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Save the plot
plot_filename = f"top_{top_n}_call_types_raw_counts_monthly_plot.png"
plt.savefig(plot_filename)
plt.close()

print(f"Plot saved as {plot_filename}")



Plot saved as top_20_call_types_raw_counts_monthly_plot.png


In [137]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter1d

# Step 1: Identify the top 20 most common call types
top_n = 20
call_type_totals = la24_results_table.group('call_type_text').select('call_type_text', 'count').sort('count', descending=True)
top_call_types = call_type_totals.take(np.arange(top_n)).column('call_type_text')

# Step 2: Categorize call types into top 20 and "Others"
la24_results_table = la24_results_table.with_column(
    'call_type_category',
    la24_results_table.apply(lambda x: x if x in top_call_types else 'Others', 'call_type_text')
)

# Step 3: Group by dispatch_date and call_type_category
grouped_data = la24_results_table.group(['dispatch_date', 'call_type_category'])

# Step 4: Smooth the data for plotting
smoothed_data = []
for call_type in grouped_data.group('call_type_category').column('call_type_category'):
    call_type_data = grouped_data.where('call_type_category', call_type)
    
    # Convert dispatch_date to datetime for grouping
    dispatch_dates = pd.to_datetime(call_type_data.column('dispatch_date'))
    counts = np.array([int(x) if x != '' else 0 for x in call_type_data.column('count')])
    
    # Smooth the counts
    smoothed_counts = gaussian_filter1d(counts, sigma=2)
    smoothed_data.append({
        'dispatch_date': dispatch_dates,
        'call_type_category': call_type,
        'smoothed_counts': smoothed_counts
    })

# Step 5: Create an interactive plot with Plotly
fig = go.Figure()

for data in smoothed_data:
    fig.add_trace(go.Scatter(
        x=data['dispatch_date'],
        y=np.log10(data['smoothed_counts'] + 1),  # Logarithmic scale for the Y-axis
        mode='lines',
        name=data['call_type_category'],
        hovertemplate=(
            "<b>Date:</b> %{x}<br>" +
            "<b>Count:</b> %{customdata}<br>" +
            "<b>Category:</b> %{text}"
        ),
        customdata=data['smoothed_counts'],  # Use raw counts for hover labels
        text=[data['call_type_category']] * len(data['dispatch_date'])  # Call type category
    ))

# Add a slider to filter the data dynamically
fig.update_layout(
    title=f"Top {top_n} Call Types Over Time (Log Scale)",
    xaxis_title="Dispatch Date",
    yaxis_title="Log10(Number of Calls + 1)",
    xaxis=dict(rangeslider=dict(visible=True)),  # Add a range slider
    legend_title="Call Type",
    template="plotly_white"
)

# Show the plot
fig.show()

### Generate Histogram for LA for hours 0-23 and density of reports for those times for entire dataset

In [138]:

import matplotlib.pyplot as plt
import numpy as np

# Step 1: Extract hours from the dispatch_time column
# Assuming dispatch_time is in the format 'HH:MM:SS' or similar
dispatch_hours = la24_results_table.apply(lambda t: int(t.split(':')[0]), 'dispatch_time')

# Step 2: Calculate the histogram for hours (0–23)
bins = np.arange(0, 25)  # Bin edges for hours 0–23
hist_counts, bin_edges = np.histogram(dispatch_hours, bins=bins, density=True)  # Normalize for relative density

# Step 3: Plot the histogram
plt.figure(figsize=(12, 6))
plt.bar(bin_edges[:-1], hist_counts, width=1, edgecolor='black', alpha=0.7)

# Step 4: Customize the plot
plt.title("Relative Density of Reports by Hour (0–23)")
plt.xlabel("Hour of the Day")
plt.ylabel("Relative Density")
plt.xticks(np.arange(0, 24, 1))  # Show all hours on the x-axis
plt.tight_layout()

# Save the plot
plot_filename = "relative_density_histogram_by_hour.png"
plt.savefig(plot_filename)
plt.close()

print(f"Histogram saved as {plot_filename}")


Histogram saved as relative_density_histogram_by_hour.png


### Now let's create overlapping distributions based on code type and area

In [139]:
import matplotlib.pyplot as plt
import numpy as np

# Step 1: Extract hours from the dispatch_time column
dispatch_hours = la24_results_table.apply(lambda t: int(t.split(':')[0]), 'dispatch_time')

# Step 2: Get unique areas
areas = np.unique(la24_results_table.column('area_occ'))

# Step 3: Plot overlapping histograms for each area
plt.figure(figsize=(12, 8))

for area in areas:
    # Filter data for the current area
    area_data = la24_results_table.where('area_occ', area)
    area_hours = area_data.apply(lambda t: int(t.split(':')[0]), 'dispatch_time')
    
    # Plot the relative density histogram as an overlapping histogram
    plt.hist(area_hours, bins=np.arange(0, 25), density=True, alpha=0.3, label=f"Area: {area}", edgecolor='black')

# Step 4: Customize the plot
plt.title("Overlapping Relative Density Histograms by Hour (Grouped by Area)")
plt.xlabel("Hour of the Day")
plt.ylabel("Relative Density")
plt.xticks(np.arange(0, 24, 1))
plt.legend(title="Areas", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Save the plot
plot_filename = "overlapping_histogram_by_area.png"
plt.savefig(plot_filename)
plt.close()

print(f"Overlapping histogram by area saved as {plot_filename}")



Overlapping histogram by area saved as overlapping_histogram_by_area.png


In [141]:
import matplotlib.pyplot as plt
import numpy as np

# Step 1: Extract hours from the dispatch_time column
dispatch_hours = la24_results_table.apply(lambda t: int(t.split(':')[0]), 'dispatch_time')

# Step 2: Identify the top 20 call type codes
top_n = 20
call_type_totals = la24_results_table.group('call_type_text').select('call_type_text', 'count').sort('count', descending=True)
top_call_types = call_type_totals.take(np.arange(top_n)).column('call_type_text')

# Step 3: Categorize codes into top 20 and "Others"
la24_results_table = la24_results_table.with_column(
    'call_type_category',
    la24_results_table.apply(lambda x: x if x in top_call_types else 'Others', 'call_type_text')
)

# Step 4: Plot overlapping histograms for each category
categories = np.unique(la24_results_table.column('call_type_category'))
plt.figure(figsize=(12, 8))

for category in categories:
    # Filter data for the current category
    category_data = la24_results_table.where('call_type_category', category)
    category_hours = category_data.apply(lambda t: int(t.split(':')[0]), 'dispatch_time')
    
    # Plot the relative density histogram as an overlapping histogram
    plt.hist(category_hours, bins=np.arange(0, 25), density=True, alpha=0.3, label=f"Call Type: {category}", edgecolor='black')

# Step 5: Customize the plot
plt.title("Overlapping Relative Density Histograms by Hour (Grouped by Call Type)")
plt.xlabel("Hour of the Day")
plt.ylabel("Relative Density")
plt.xticks(np.arange(0, 24, 1))
plt.legend(title="Call Types", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# Save the plot
plot_filename = "overlapping_histogram_by_call_type.png"
plt.savefig(plot_filename)
plt.close()

print(f"Overlapping histogram by call type saved as {plot_filename}")


Overlapping histogram by call type saved as overlapping_histogram_by_call_type.png


### Create Correlation Matrix for valuable columns from the table

In [142]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assume `la24_results_df` is your DataFrame
# Columns: 'dispatch_date', 'dispatch_time', 'count', 'area_occ', 'call_type_code'

# Step 1: Convert 'dispatch_date' to a proper datetime object
la24_results_df['dispatch_date'] = pd.to_datetime(la24_results_df['dispatch_date'])

# Step 2: Extract hour from 'dispatch_time' (assumes HH:MM:SS format)
la24_results_df['hour'] = la24_results_df['dispatch_time'].str.split(':').str[0].astype(int)

# Step 3: Extract day of the week from 'dispatch_date'
la24_results_df['day_of_week'] = la24_results_df['dispatch_date'].dt.dayofweek  # Monday=0, Sunday=6

# Step 4: Convert string columns to numeric
la24_results_df['area_encoded'] = pd.factorize(la24_results_df['area_occ'])[0]  # Encode 'area_occ'
la24_results_df['call_type_code_encoded'] = pd.factorize(la24_results_df['call_type_text'])[0]  # Encode 'call_type_code'

# Step 5: Select relevant columns for correlation
columns_for_correlation = ['hour', 'day_of_week', 'area_encoded', 'call_type_code_encoded']

# Compute correlation matrix
correlation_matrix = la24_results_df[columns_for_correlation].corr()

# Step 6: Visualize correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix Including Areas and Call Types")

# Save the heatmap as a PNG file
plot_filename = "correlation_matrix_with_call_types_and_areas.png"
plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
plt.close()

print(f"Correlation matrix heatmap saved as {plot_filename}")


Correlation matrix heatmap saved as correlation_matrix_with_call_types_and_areas.png


### ANOVA Test on Call Volumes by Hour per Day 

#### null: no differece of call volumes depending on hour of day
#### alt: significant differnce between call volumes depending on hour of day

In [143]:
from scipy.stats import f_oneway
import pandas as pd

# Group by dispatch_date and hour
hourly_counts = la24_results_df.groupby(['dispatch_date', 'hour']).size().reset_index(name='call_count')

# Prepare data for ANOVA (group by hour)
hourly_groups = [hourly_counts[hourly_counts['hour'] == h]['call_count'].values for h in range(24)]

# Remove empty groups (hours with no data)
hourly_groups = [group for group in hourly_groups if len(group) > 1]

# Perform ANOVA
f_stat, p_value = f_oneway(*hourly_groups)
print(f"ANOVA F-statistic: {f_stat}, p-value: {p_value}")



ANOVA F-statistic: 1091.8449810545758, p-value: 0.0


### Perform Tukey HSD test to compare all categories to see which ones are signifcant against one another

In [144]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pandas as pd

# Assume `hourly_counts` is the DataFrame from ANOVA setup
# Columns: 'dispatch_date', 'hour', 'call_count'

# Flatten the data for Tukey's test
hourly_counts = la24_results_df.groupby(['dispatch_date', 'hour']).size().reset_index(name='call_count')

# Perform Tukey's HSD test
tukey = pairwise_tukeyhsd(
    endog=hourly_counts['call_count'],  # Dependent variable
    groups=hourly_counts['hour'],      # Grouping variable
    alpha=0.05                         # Significance level
)

# Print results
print(tukey)

tukey_results_df = pd.DataFrame(data=tukey.summary().data[1:], columns=tukey.summary().data[0])

# Plot Tukey results as a table
plt.figure(figsize=(12, 8))
plt.axis('off')
plt.table(cellText=tukey_results_df.values, colLabels=tukey_results_df.columns, loc='center')
plt.title("Tukey HSD Test Results")
plt.savefig("tukey_test_results.png", dpi=300, bbox_inches='tight')
plt.close()

tukey.plot_simultaneous()
plt.title("Tukey HSD Test Results")
plt.savefig("tukey_test_simultaneous_plot.png", dpi=300, bbox_inches='tight')
plt.close()


 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     0      1 -26.9233    0.0 -34.4205 -19.4261   True
     0      2 -49.2148    0.0 -56.7172 -41.7125   True
     0      3 -73.9836    0.0 -81.4808 -66.4864   True
     0      4  -89.074    0.0 -96.5712 -81.5768   True
     0      5 -78.0493    0.0 -85.5465 -70.5521   True
     0      6 -53.3945    0.0 -60.8917 -45.8973   True
     0      7  18.1616    0.0  10.6645  25.6588   True
     0      8  28.3973    0.0  20.9001  35.8945   True
     0      9  38.5918    0.0  31.0946   46.089   True
     0     10  38.3479    0.0  30.8508  45.8451   True
     0     11  43.7945    0.0  36.2973  51.2917   True
     0     12  35.4904    0.0  27.9932  42.9876   True
     0     13  45.3342    0.0  37.8371  52.8314   True
     0     14  31.9151    0.0  24.4179  39.4123   True
     0     15  34.9644    0.0  27.4672  42.4616   True
     0    

### Chi Square Test 

#### null: no association between call types and areas
#### alt: association between call types and areas

In [145]:
from scipy.stats import chi2_contingency

# Create contingency table
contingency_table = pd.crosstab(la24_results_df['call_type_code'], la24_results_df['area_occ'])

# Perform Chi-Square Test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Display Results
print(f"Chi-Square Statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies:")
print(expected)

# Interpretation
if p < 0.05:
    print("There is a significant association between call type and area.")
else:
    print("No significant association between call type and area.")

# Create DataFrames for observed and expected frequencies
observed_df = pd.DataFrame(contingency_table)
expected_df = pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns)

# Save observed frequencies
plt.figure(figsize=(10, 6))
plt.axis('off')
plt.table(cellText=observed_df.values, colLabels=observed_df.columns, rowLabels=observed_df.index, loc='center')
plt.title("Observed Frequencies: Call Type vs. Area")
plt.savefig("chi_square_observed.png", dpi=300, bbox_inches='tight')
plt.close()

# Save expected frequencies
plt.figure(figsize=(10, 6))
plt.axis('off')
plt.table(cellText=expected_df.values, colLabels=expected_df.columns, rowLabels=expected_df.index, loc='center')
plt.title("Expected Frequencies: Call Type vs. Area")
plt.savefig("chi_square_expected.png", dpi=300, bbox_inches='tight')
plt.close()

# Observed heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(contingency_table, cmap="coolwarm", annot=False, cbar=True)
plt.title("Observed Frequencies: Call Type vs. Area")
plt.xlabel("Area")
plt.ylabel("Call Type")
plt.savefig("chi_square_observed_heatmap.png", dpi=300, bbox_inches='tight')
plt.close()

# Expected heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(expected_df, cmap="coolwarm", annot=False, cbar=True)
plt.title("Expected Frequencies: Call Type vs. Area")
plt.xlabel("Area")
plt.ylabel("Call Type")
plt.savefig("chi_square_expected_heatmap.png", dpi=300, bbox_inches='tight')
plt.close()

Chi-Square Statistic: 1040233.8803438479
P-value: 0.0
Degrees of Freedom: 18900
Expected Frequencies:
[[  2.40363575e+04   2.47219761e+04   1.70426378e+04 ...,   1.77434015e+04
    1.67114943e+04   1.97650556e+04]
 [  6.51446971e-01   6.70028996e-01   4.61899222e-01 ...,   4.80891715e-01
    4.52924381e-01   5.35683728e-01]
 [  7.19848903e+01   7.40382041e+01   5.10398640e+01 ...,   5.31385345e+01
    5.00481441e+01   5.91930520e+01]
 ..., 
 [  1.34360938e+00   1.38193480e+00   9.52667145e-01 ...,   9.91839163e-01
    9.34156536e-01   1.10484769e+00]
 [  9.77170457e-01   1.00504349e+00   6.92848832e-01 ...,   7.21337573e-01
    6.79386571e-01   8.03525593e-01]
 [  4.07154357e-02   4.18768123e-02   2.88687014e-02 ...,   3.00557322e-02
    2.83077738e-02   3.34802330e-02]]
There is a significant association between call type and area.


### Use Linear Regression to Predict Call Volumes by Hour

In [149]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Prepare data for modeling
la24_results_df['hour'] = la24_results_df['dispatch_time'].str.split(':').str[0].astype(int)

count_by_hour = la24_results_df.groupby('hour').size().reset_index(name='count')

x = count_by_hour[['hour']]
y = count_by_hour['count']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-squared:", r2_score(y_test, y_pred))

# Plot predictions against the data
plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test, color='blue', label='Actual Data')
plt.scatter(X_test, y_pred, color='red', label='Predictions')
plt.plot(X_test, y_pred, color='red', linestyle='dashed', label='Regression Line')
plt.xlabel('Hour')
plt.ylabel('Count')
plt.title('Actual vs Predicted Count by Hour')
plt.legend()
plt.grid(True)

# Save the plot as a PNG file
plot_path = "call_vol_by_time_predictions_chart.png"
plt.savefig(plot_path)
plt.close()

print(f"Chart saved to: {plot_path}")


Mean Squared Error: 189290388.359
R-squared: -4.502481970336834
Chart saved to: call_vol_by_time_predictions_chart.png


### Forecast call volumes per hour or day

In [151]:
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt

# Prepare data for Prophet
time_series = la24_results_df.groupby('dispatch_date').size().reset_index(name='count')
time_series.rename(columns={'dispatch_date': 'ds', 'count': 'y'}, inplace=True)

# Fit a Prophet model
model = Prophet()
model.fit(time_series)

# Make future predictions
future = model.make_future_dataframe(periods=30)  # Extend for 30 days
forecast = model.predict(future)

# Plot the forecast
forecast_fig = model.plot(forecast)
plt.title("Call Volume Forecast")
plt.xlabel("Date")
plt.ylabel("Call Volume")

# Save the plot as a PNG file
forecast_plot_filename = "call_volume_forecast.png"
forecast_fig.savefig(forecast_plot_filename, dpi=300, bbox_inches='tight')

print(f"Forecast plot saved as {forecast_plot_filename}")



DEBUG:cmdstanpy:cmd: where.exe tbb.dll
cwd: None
DEBUG:cmdstanpy:TBB already found in load path
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: C:\Users\wolfe\AppData\Local\Temp\tmpya7fr0s5\esrf9ugz.json
DEBUG:cmdstanpy:input tempfile: C:\Users\wolfe\AppData\Local\Temp\tmpya7fr0s5\qozma2qe.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['C:\\Users\\wolfe\\OneDrive\\Desktop\\ASAdatachallenge24\\venv\\Lib\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=29122', 'data', 'file=C:\\Users\\wolfe\\AppData\\Local\\Temp\\tmpya7fr0s5\\esrf9ugz.json', 'init=C:\\Users\\wolfe\\AppData\\Local\\Temp\\tmpya7fr0s5\\qozma2qe.json', 'output', 'file=C:\\Users\\wolfe\\AppData\\Local\\Temp\\tmpya7fr0s5\\prophet_modelcv28392z\\prophet_model-2

Forecast plot saved as call_volume_forecast.png


### Introducing ACS Data 2022 for CA (LA County) to Pair with Call Service Data

In [ ]:
import pandas as pd
import geopandas as gpd
from getpass import getpass
import plotly.express as px
import requests
import warnings

pd.set_option('display.max_columns', None) # show all columns
warnings.filterwarnings("ignore") # suppress warnings

census_api_key = ""


In [167]:
def json_to_df(response):
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [168]:
url = "https://api.census.gov/data/2022/pep/charage?get=NAME,POP,AGE,DATE_CODE,DATE_DESC&for=state:06&key={0}".format(census_api_key)
response = requests.request("GET", url)
response
census_df = json_to_df(response)
#census_df['POP'] = census_df['POP'].astype(int)

#census_df.head(5)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
census_df.describe